In [1]:
#%pip install plotly
import plotly.graph_objects as go
import pandas as pd

# Read TSV file into DataFrame
df_one = pd.read_csv('../data/smolvlm_results_combined_1.tsv', sep='\t')
df_two = pd.read_csv('../data/smolvlm_m1_sft_results_0_to_5000.tsv', sep='\t')

# df_one = pd.read_csv('../data/smolvlm_m2_baseline.tsv', sep='\t')
# df_two = pd.read_csv('../data/smolvlm_m2_sft_results_0_to_5000.tsv', sep='\t')

# Display basic info about the DataFrame
print(f"Shape: {df_one.shape}")
print(f"Columns: {list(df_one.columns)}")
print("\nFirst few rows:")
display(df_one.head())

print(f"Shape: {df_two.shape}")
print(f"Columns: {list(df_two.columns)}")
print("\nFirst few rows:")
display(df_two.head())

Shape: (5000, 5)
Columns: ['index', 'prompt1', 'prompt2', 'prompt3', 'prompt4']

First few rows:


,index,prompt1,prompt2,prompt3,prompt4
0,0,In this image we can see a boy holding an umbr...,No.,Female.,Male.
1,1,"A kitchen is shown, with a white door on the r...",No.,Female.,Female.
2,2,In this image we can see a girl holding a cat ...,Yes.,Female.,Female.
3,3,"In this image we can see a toilet, a bottle, a...",No.,Female.,Female.
4,4,"A room has two sinks, a fire extinguisher, and...",No.,NO.,Female.


Shape: (5000, 5)
Columns: ['index', 'prompt1', 'prompt2', 'prompt3', 'prompt4']

First few rows:


,index,prompt1,prompt2,prompt3,prompt4
0,0,"In the image, there are two large cows standin...",Yes.,Female.,Female.
1,1,The image depicts a kitchen with a white count...,"No, there is no person in this image.",Female.,Female.
2,2,A young girl is holding a small kitten in her ...,Yes.,Female.,Female.
3,3,"A bathroom with a toilet, a toilet brush, and ...","No, there is no person in this image.",Male.,Male.
4,4,The image depicts a room with a mix of modern ...,"No, there is no person in this image.",Male.,Female.


In [2]:
comparison_df = pd.DataFrame({
    'index': df_one.index,
    'prompt2_df_one': df_one['prompt2'],
    'prompt2_df_two': df_two['prompt2'],
    'prompt3_df_one': df_one['prompt3'],
    'prompt3_df_two': df_two['prompt3']
})

print("\nComparison of differing rows:")
display(comparison_df)


Comparison of differing rows:


,index,prompt2_df_one,prompt2_df_two,prompt3_df_one,prompt3_df_two
0,0,No.,Yes.,Female.,Female.
1,1,No.,"No, there is no person in this image.",Female.,Female.
2,2,Yes.,Yes.,Female.,Female.
3,3,No.,"No, there is no person in this image.",Female.,Male.
4,4,No.,"No, there is no person in this image.",NO.,Male.
...,...,...,...,...,...
4995,4995,No.,"No, there is no person in this image.",Female.,Female.
4996,4996,Yes.,Yes.,Male.,Male.
4997,4997,NO.,Yes.,Female.,Female.
4998,4998,No.,"No, there is no person in this image.",Female.,Male.


In [3]:
# Read all labels
with open('../data/labels/yes_labels.txt', 'r') as file:
    yes_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/no_labels.txt', 'r') as file:
    no_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/other_labels_yn.txt', 'r') as file:
    other_labels_yn = [line.strip() for line in file.readlines()]

with open('../data/labels/male_labels.txt', 'r') as file:
    male_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/female_labels.txt', 'r') as file:
    female_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/other_labels_mf.txt', 'r') as file:
    other_labels_mf = [line.strip() for line in file.readlines()]

In [4]:
def replace_label(prompt_text, yn=True):
    if yn:
        if prompt_text in yes_labels:
             return 'yes'
        elif prompt_text in no_labels:
            return 'no'
        elif prompt_text in other_labels_yn:
            return 'other'
        else:
            print(f"ALERT: {prompt_text} not found in yes/no labels")
            return prompt_text
    else:
        if prompt_text in male_labels:
             return 'male'
        elif prompt_text in female_labels:
            return 'female'
        elif prompt_text in other_labels_mf:
            return 'other'
        else:
            print(f"ALERT: {prompt_text} not found in male/female labels")
            return prompt_text

# Apply the function to replace labels
comparison_df['prompt2_df_one'] = comparison_df['prompt2_df_one'].apply(replace_label)
comparison_df['prompt2_df_two'] = comparison_df['prompt2_df_two'].apply(replace_label)
comparison_df['prompt3_df_one'] = comparison_df['prompt3_df_one'].apply(replace_label, yn=False)
comparison_df['prompt3_df_two'] = comparison_df['prompt3_df_two'].apply(replace_label, yn=False)

In [5]:
def assign_group1(row, first=True):
    if first:
        prompt2 = row['prompt2_df_one']
        prompt3 = row['prompt3_df_one']
    else:
        prompt2 = row['prompt2_df_two']
        prompt3 = row['prompt3_df_two']
    
    transition = f"{prompt2} & {prompt3}"
    
    group_mapping = {
       'yes & male': 1,
       'yes & female': 2,
       'yes & other': 3,
       'no & male': 4,
       'no & female': 5,
       'no & other': 6,
       'other & male': 7,
       'other & female': 8,
       'other & other': 9,
    }
    return group_mapping.get(transition, None)

# Apply the function to create the yn_groups column
comparison_df['group_one'] = comparison_df.apply(assign_group1, axis=1)
comparison_df['group_two'] = comparison_df.apply(assign_group1, axis=1, first=False)
comparison_df.head()

,index,prompt2_df_one,prompt2_df_two,prompt3_df_one,prompt3_df_two,group_one,group_two
0,0,no,yes,female,female,5,2
1,1,no,no,female,female,5,5
2,2,yes,yes,female,female,2,2
3,3,no,no,female,male,5,4
4,4,no,no,other,male,6,4


In [6]:
# Create reverse mapping for labels
reverse_mapping = {
    1: 'yes & male',
    2: 'yes & female', 
    3: 'yes & other',
    4: 'no & male',
    5: 'no & female',
    6: 'no & other',
    7: 'other & male',
    8: 'other & female',
    9: 'other & other'
}

# Create meaningful labels in consistent order with bold formatting
pre_labels = [f"<b>{reverse_mapping[i]}</b>" for i in range(1, 10)]
post_labels = [f"<b>{reverse_mapping[i]}</b>" for i in range(1, 10)]
all_labels = pre_labels + post_labels

# Define academic color scheme - same colors for matching labels
base_colors = [
    '#1f77b4',  # yes & male - Professional Blue
    '#ff7f0e',  # yes & female - Professional Orange
    '#2ca02c',  # yes & other - Professional Green
    '#d62728',  # no & male - Professional Red
    '#9467bd',  # no & female - Professional Purple
    '#8c564b',  # no & other - Professional Brown
    '#e377c2',  # other & male - Professional Pink
    '#7f7f7f',  # other & female - Professional Gray
    '#bcbd22'   # other & other - Professional Olive
]

# Use the same colors for both left and right nodes
node_colors = base_colors + base_colors

# FIXED: Create aggregated data first, then extract source, target, and values
grouped_data = comparison_df.groupby(['group_one', 'group_two']).size().reset_index(name='count')

In [7]:
# Now create aligned arrays
source = grouped_data['group_one'].values - 1  # Convert to 0-based indexing
target = grouped_data['group_two'].values + 8  # Offset targets to positions 9-17
value = grouped_data['count'].values

# Create link colors based on source nodes
link_colors = []
for src in source:
    color = node_colors[src]
    # Convert hex to rgba with transparency
    if color.startswith('#'):
        hex_color = color[1:]
        rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        link_colors.append(f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, 0.4)')

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = all_labels,
        color = node_colors
    ),
    link = dict(
        source = source,
        target = target,
        value = value,
        color = link_colors
    )
)])

# Update layout for better appearance
fig.update_layout(
    title="SmolVLM 256M Baseline to SFT Response Transition Analysis",
    font_size=12,
    font_family="Arial",
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()